In [1]:
import torch
import torch.nn as nn
from torchinfo import summary

In [2]:
class LinearDilationCNN(nn.Module):
    def __init__(self, num_layers):
        super(LinearDilationCNN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            dilation = i + 1
            self.layers.append(nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, dilation=dilation))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def calc_receptive_field(self):
        receptive_field = 1
        for layer in self.layers:
            dilation = layer.dilation[0]
            kernel_size = layer.kernel_size[0]
            receptive_field += (kernel_size - 1) * dilation
        return receptive_field


In [3]:
class ExponentialDilationCNN(nn.Module):
    def __init__(self, num_layers):
        super(ExponentialDilationCNN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            dilation = 2 ** i
            self.layers.append(nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, dilation=dilation))

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def calc_receptive_field(self):
        receptive_field = 1
        for layer in self.layers:
            dilation = layer.dilation[0]
            kernel_size = layer.kernel_size[0]
            receptive_field += (kernel_size - 1) * dilation
        return receptive_field


In [4]:
# Define a dummy input to print the model summary
dummy_input = torch.randn(1, 1, 512)

## CNN with Linear Dilations

In [5]:
model_lin = LinearDilationCNN(num_layers=4)

summary_lin = summary(model_lin, input_size=dummy_input.shape, depth=4, verbose=1)

Layer (type:depth-idx)                   Output Shape              Param #
LinearDilationCNN                        [1, 1, 492]               --
├─ModuleList: 1-1                        --                        --
│    └─Conv1d: 2-1                       [1, 1, 510]               4
│    └─Conv1d: 2-2                       [1, 1, 506]               4
│    └─Conv1d: 2-3                       [1, 1, 500]               4
│    └─Conv1d: 2-4                       [1, 1, 492]               4
Total params: 16
Trainable params: 16
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.02


In [6]:
lin_receptive_field = model_lin.calc_receptive_field()
print(f"Linear Model Receptive Field: {lin_receptive_field}\n\n")

for i in range(4):
    print(f"Linear Model Layer {i + 1} - Dilation: {i + 1}, Receptive Field: {model_lin.layers[i].dilation[0]}\n")

Linear Model Receptive Field: 21


Linear Model Layer 1 - Dilation: 1, Receptive Field: 1

Linear Model Layer 2 - Dilation: 2, Receptive Field: 2

Linear Model Layer 3 - Dilation: 3, Receptive Field: 3

Linear Model Layer 4 - Dilation: 4, Receptive Field: 4



In [7]:
model_exp = ExponentialDilationCNN(num_layers=4)

summary_exp = summary(model_exp, input_size=dummy_input.shape, depth=4, verbose=1)

Layer (type:depth-idx)                   Output Shape              Param #
ExponentialDilationCNN                   [1, 1, 482]               --
├─ModuleList: 1-1                        --                        --
│    └─Conv1d: 2-1                       [1, 1, 510]               4
│    └─Conv1d: 2-2                       [1, 1, 506]               4
│    └─Conv1d: 2-3                       [1, 1, 498]               4
│    └─Conv1d: 2-4                       [1, 1, 482]               4
Total params: 16
Trainable params: 16
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.02


In [8]:
exp_receptive_field = model_exp.calc_receptive_field()
print(f"Exponential Model Receptive Field: {exp_receptive_field}\n\n")

for i in range(4):
    print(f"Exponential Model Layer {i + 1} - Dilation: {2**i}, Receptive Field: {model_exp.layers[i].dilation[0]}\n")

Exponential Model Receptive Field: 31


Exponential Model Layer 1 - Dilation: 1, Receptive Field: 1

Exponential Model Layer 2 - Dilation: 2, Receptive Field: 2

Exponential Model Layer 3 - Dilation: 4, Receptive Field: 4

Exponential Model Layer 4 - Dilation: 8, Receptive Field: 8

